# F5.news Trending News - Machine Learning Exploration

- News Article Sentiment
- Predict Trending Topics
- Topic Categorization

### Installs & Imports

In [ ]:
%pip install -q -U boto3 hvac mlflow numpy "pyspark==3.5.0" python-dotenv "pymongo[srv]"

In [ ]:
import os
import hvac
import mlflow
import numpy as np

from dotenv import load_dotenv
from datetime import datetime, timedelta

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import Bucketizer, StringIndexer, VectorAssembler, RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import udf
from pyspark.sql.functions import col

from mlflow.models import infer_signature

load_dotenv()

### Connect to Vault for Mongo Secrets

In [ ]:
client = hvac.Client(
    url=os.environ.get('VAULT_ADDR'),
    token=os.environ.get('VAULT_TOKEN'),
)

print(client.is_authenticated())

if client.is_authenticated():
    try:
        secret_resp = client.secrets.kv.v2.read_secret_version(
            mount_point='kv', 
            path='f5.news', 
            raise_on_deleted_version=False
        )
        
        if secret_resp['data'] is not None:
            secret_values = secret_resp['data']['data']
            for secret, value in secret_values.items():
                os.environ[str(secret)] = str(value)
        else:
            print("The secret does not exist.")
    except hvac.exceptions.InvalidPath:
        print("The path is invalid or the permission is denied.")
    except hvac.exceptions.Forbidden:
        print("The permission is denied.")
    except hvac.exceptions.VaultError as e:
        print(f"Vault error occurred: {e}")
else:
    print("Failed to connect to HashiVault")

### Configs

In [ ]:
# General
DEBUG = False
MODE = "local" # Supported -- local OR cluster
REG_PARAM_VALUE = 0.1 # Experimenting with this value can improve final accuracy
MAX_ITER = 20
DATASET_SPLIT = [0.9, 0.1] # Portion of data to split between training and test datasets
SAMPLE_TITLE_1 = "Trump Says Some Migrants Are ‘Not People’ and Predicts a ‘Blood Bath’ if He Loses"
SAMPLE_TITLE_2 = "California Insurance Commissioner Ricardo Lara responds after State Farm announces it will not renew thousands of policies - ABC7 Los Angeles"
SAMPLE_TITLE_3 = "Oklahoma students walk out after trans student's death to protest bullying policies"

# Spark
SPARK_MASTER = "spark://localhost:7077"
SPARK_MEMORY = "4g"
os.environ["PYSPARK_PIN_THREAD"] = "false" # TODO: Move to .env

# Mongo
URI = os.environ['mongo_uri']
DATABASE = os.environ['database']
COLLECTION = os.environ['collection']

# MLflow
MLFLOW_API = "http://localhost:5000"
MODEL_NAME = "f5news_upvote_bucket_prediction"
EXPERIMENT_NAME = "f5news_upvote_bucket_prediction"

# Minio S3
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://localhost:9000"
os.environ['AWS_ACCESS_KEY_ID'] = "minio"
os.environ['AWS_SECRET_ACCESS_KEY'] = "minio123" # TODO: Move all of these to .env

### Pull F5 records using pymongo client

In [ ]:
if DEBUG:
    # Create a new client and connect to the MongoDB server
    client = MongoClient(URI, server_api=ServerApi('1'))

    # Send a ping to confirm a successful connection
    try:
        client.admin.command('ping')
        print("Successfully connected to MongoDB...")
    except Exception as e:
        print(e)

    try:
        database = client[DATABASE]
        collection = database[COLLECTION]

        # Query all documents in the collection
        documents = collection.find({"sub": "politics"}).sort({"upvoteCount": -1, "fetchedAt": -1})

        if(DEBUG == True):
            # Iterate over the cursor to access the documents
            for doc in documents:
                print(doc["title"])
                print(doc["fetchedAt"])
                print(doc["upvoteCount"], "upvotes")
                print()
        else:
            print("Mongo documents loaded successfully!")
    except Exception as e:
        print(e)

### Setup MLflow runner

In [ ]:
global_run_name = None
global_run_id = None
start_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Set MLflow configs
mlflow.set_tracking_uri(MLFLOW_API)
mlflow.set_experiment(EXPERIMENT_NAME)

def start_mlflow_run(run_name: str = None):
    global global_run_name, global_run_id, start_time
    if run_name is None:
        run_name = start_time
    else:
        run_name = run_name + start_time
    global_run_name = run_name
    run = mlflow.start_run(run_name=run_name, description=EXPERIMENT_NAME)
    global_run_id = run.info.run_id

### Connect to Spark

In [ ]:
# Create MLflow Run Instance
try:
    mlflow.end_run()
except:
    pass

start_mlflow_run()

# Log parameters
start_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
mlflow.log_param("start_time", start_time)

try:
    if MODE == "local":
        print("Starting Spark in local mode")
        spark = SparkSession.builder \
            .appName("F5news") \
            .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1,org.mlflow:mlflow-spark:2.8.1") \
            .getOrCreate()
    elif MODE == "cluster":
        print("Starting Spark in cluster mode")
        spark = SparkSession.builder \
            .appName("F5news") \
            .master(SPARK_MASTER) \
            .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1,org.mlflow:mlflow-spark:2.8.1") \
            .getOrCreate()
    
    # Setup Spark AutoLog
    mlflow.autolog()

    # Get Spark version
    spark_version = spark.version
    print("Spark Version:", spark_version)

    # Check if the master URL indicates local mode or a specific cluster mode
    sc = spark.sparkContext
    master_url = sc.master
    
    if "local" and not "localhost" in master_url:
        print("PySpark is running in local mode.")
    else:
        print("PySpark is running in cluster mode with master URL:", master_url)

except Exception as e:
    print("Error:", str(e))

    # Stop SparkSession
    spark.stop()

    # End MLflow run
    mlflow.end_run()

### Load MongoDB as Dataset

In [ ]:
# Declare Schema
schema = StructType([
    StructField("title", StringType(), nullable=True), # model input
    StructField("upvoteCount", DoubleType(), nullable=True), # used to bucketize for training
    StructField("fetchedAt", TimestampType(), nullable=True) # used to filter recent events
])

# Load data from MongoDB into a DataFrame
df = spark.read \
    .format("mongo") \
    .option("uri", URI) \
    .option("database", DATABASE) \
    .option("collection", COLLECTION) \
    .schema(schema) \
    .load()

print("Data loaded successfully from MongoDB!")

### Show Loaded Data

In [ ]:
if DEBUG:
    df.show(5,truncate=False)

### Filter Out Recent Posts

In [ ]:
# Get document initial count
print('Documents Loaded:', df.count())
mlflow.log_param("loaded_documents", df.count())

# Filter out new posts
oneDayAgo = d = datetime.today() - timedelta(days=1)
df = df.filter(df.fetchedAt < oneDayAgo)
print('Total Filtered Documents:', df.count())

mlflow.log_param("filtered_documents", df.count())

### Bucketize Input Dataset

In [ ]:
bucketizer = Bucketizer(splits=[0, 1000, 5000, 10000, 25000, 50000, float('inf')], inputCol="upvoteCount", outputCol="bucket")
df = bucketizer.transform(df)

### Preview a Random Sample of Bucketized Dataset

In [ ]:
if DEBUG:
    sample_count = 10
    pandas_random_sample = df.toPandas().sample(n=sample_count) # Convert to pandas dataframe to take sample
    pyspark_random_sample = spark.createDataFrame(pandas_random_sample) # Convert back to pyspark dataframe
    pyspark_random_sample.show()
    df.printSchema()

### Drop Unnecessary Columns

In [ ]:
df.drop('upvoteCount')
df.drop('fetchedAt')

### Split Dataset into Training and Test

In [ ]:
(trainingData, testData) = df.randomSplit(DATASET_SPLIT, seed = 123456)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

mlflow.log_metric("trainingData", trainingData.count())
mlflow.log_metric("testData", testData.count())

### Define model Pipeline Steps

- **Regular Expression Tokenizer**: Breaks title into array of words via regex
- **Stop Words Remover**: Removes undesireable words from Regex Tokenizer output
- **Bag of Words Counter**: Creates vector representation of the array of words extracted from original title string
- **Logistic Regression**: Trains logistic regression to classify 'bucket' based on the features returned by 'Bag of words' counter/features column.

In [ ]:
# Tokenizer
regexTokenizer = RegexTokenizer(inputCol="title", outputCol="words", pattern="\\W")

# StopWordsRemover
add_stopwords = ["http","https","amp","reddit","subreddit"] # TODO: Update stopwords to match dataset
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)

# CountVectorizer
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=30000, minDF=5)

# Init linear regression model with column names
lr = LogisticRegression(featuresCol="features", labelCol="bucket")

# Create ParamGrid for Cross Validation
# paramGrid = (
#     ParamGridBuilder()
#     .addGrid(lr.regParam, [0.1, 0.3, 0.5]) # regularization parameter
#     .addGrid(lr.elasticNetParam, [0.0, 0.1, 0.2]) # Elastic Net Parameter (Ridge = 0)
#     .addGrid(lr.maxIter, [10, 20, 50]) #Number of iterations
#     .build()
# )

# define evaluator for cross validator
# evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")

# Create 5-fold CrossValidator
# cv = CrossValidator(
#    estimator=lr, \
#    estimatorParamMaps=paramGrid, \
#    evaluator=evaluator, \
#    numFolds=5
# )

### Assemble Model Pipeline

Creates the `features` columns. We split titles to words, remove the words we don't want, vectorize the resulting array of words, then label based on bucket column.

In [ ]:
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, lr])

### Train the Model

In [ ]:
model = pipeline.fit(trainingData) # This step is what actually does the training

### Evaluate the Model Using Test Data

In [ ]:
# Make Predictions for entire test data set
predictions = model.transform(testData)

# Show a few predictions
# - change filter params such as prediction == 1 # TODO: Document what this does
if DEBUG:
    display(predictions.select("title","bucket","probability","prediction","features").orderBy("probability", ascending=False).toPandas().sample(n=10))

# Calculate & Log RMSE
rmse = predictions.selectExpr("sqrt(avg(pow(bucket - prediction, 2))) as RMSE").collect()[0]["RMSE"]
print("Root Mean Squared Error (RMSE) on Test Data:", rmse) # TODO: Determine output label
mlflow.log_metric("rmse", rmse)

# Calculate & Log Accuracy
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="bucket")
lr_accuracy = evaluator.evaluate(predictions)
print("Logistical Regression Accuracy:", lr_accuracy)
mlflow.log_metric("lr_accuracy", lr_accuracy)

### Log Final Model to MLflow

In [ ]:
try:
    mlflow.spark.log_model(
        spark_model = model, 
        artifact_path = "model",
    #    signature = signature,
        registered_model_name = "f5news_upvote_bucket_prediction",
    )
except Exception as e:
    print(f"[Warning] Unable to save the model to MLflow: {e}")
    pass

### Save the Final Model to Disk

In [ ]:
try: 
    top_level_dir = "models"
    os.makedirs(top_level_dir, exist_ok=True)

    model_dir = os.path.join(top_level_dir, EXPERIMENT_NAME)
    os.makedirs(model_dir, exist_ok=True)

    model.save(os.path.join(model_dir, start_time))
except Exception as e:
    print(f"Error saving the model to disk: {e}")
    pass

## Use the Model

### Prepare Prediction Data

In [ ]:
testInput = spark.createDataFrame([
    {'title': SAMPLE_TITLE_1},
    {'title': SAMPLE_TITLE_2},
    {'title': SAMPLE_TITLE_3}
])

testInput.show()

### Load the Model & Make Predictions

- **Bucket 0**: 0 - 999 upvotes
- **Bucket 1**: 1,000 - 4,999 upvotes
- **Bucket 2**: 5,000 - 9,999 upvotes
- **Bucket 3**: 10,000 - 24,999 upvotes
- **Bucket 4**: 25,000 - 49,999 upvotes
- **Bucket 5**: > 50,000 upvotes

In [ ]:
model = mlflow.spark.load_model(f'runs:/{global_run_id}/model') # global_run_id is set by when an MLflow run is initiated

predictions = model.transform(testInput)
predictions.show()

### Extract Predictions

In [ ]:
confidence_scores = predictions.select("probability").rdd.map(lambda x: x[0][1]).collect()
formatted_scores = ["{:.2f}%".format(score * 100) for score in confidence_scores]

print(formatted_scores)

### Close Out Sessions

In [ ]:
# Stop SparkSession
try:
    spark.stop()
except:
    pass

# End MLflow run
mlflow.end_run()